Hello everyone!

This is the notebook for our final project! I'm committing this file to the repo so that everyone can start from the same spot.

Few questions:

1) Video call meetups?

2) Regarding Git/GitHub, how comfortable are you all with checking out your own branches to work and then pushing the changes to the remote master branch?